# Reloading the stored model

In [16]:
import mlflow
import pandas as pd
import joblib
import mlflow
from mlflow.models.signature import infer_signature


# load data and model
x = pd.read_csv('../data/adult_test.csv').drop(['income'], axis=1)
# print(x[:10])
# x.describe()
# x.dtypes
x['age']=x['age'].astype(int)
x['hours_per_week']=x['hours_per_week'].astype(int)
x['capital_gain']=x['capital_gain'].astype(int)
x['capital_loss']=x['capital_loss'].astype(int)

clf = joblib.load('../model.pkl')

In [18]:
# removing the best model folder if already present
!rm -rf ./best_model

In [19]:
signature = infer_signature(x, clf.predict(x))
input_example = {}
for i in x.columns:
    input_example[i] = x[i][0]

print(input_example)

mlflow.sklearn.save_model(clf, "best_model", signature=signature, input_example=input_example)

{'age': 39, 'work_class': 'Local-gov', 'education': 'Masters', 'marital_status': 'Married', 'occupation': 'Professional', 'relationship': 'Wife', 'race': 'White', 'sex': 'Female', 'capital_gain': 0, 'capital_loss': 0, 'hours_per_week': 28, 'native_country': 'United-States'}


/home/manne/bin/anaconda3/envs/msbd/lib/python3.8/site-packages/mlflow/models/signature.py:123: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input)


In [20]:
saved_model_path="./best_model"
reloaded_model = mlflow.sklearn.load_model(model_uri=saved_model_path)

In [21]:
test_csv_path = '../data/adult_test.csv'
test_dataset = pd.read_csv(test_csv_path)
target_column = 'income'
y_test = test_dataset[target_column]
x_test = test_dataset.drop(target_column, axis=1)


In [22]:
print(reloaded_model.score(x_test,y_test))

0.842814667988107


# Creating a docker container from the stored model

We can build a docker image containing the model using mlflow's cli:

!mlflow models build-docker -m "./best_model" -n "random_forest_adult"


In [4]:
#docker run -p 5001:8080 random_forest_adult

In [4]:
import requests
import json


with open('./best_model/input_example.json') as f:
    sample_input = json.load(f)

#sample_input['data'][0][0]=50 # changing age value
#sample_input['data'][0][3]='Married' # changing age value
#sample_input['data'][0][5]='Husband' # changing age value
print(sample_input)

{'columns': ['age', 'work_class', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country'], 'data': [[39, 'Local-gov', 'Masters', 'Married', 'Professional', 'Wife', 'White', 'Female', 0, 0, 28, 'United-States']]}


In [5]:
response = requests.post(
              url='http://127.0.0.1:5001/invocations', data=json.dumps(sample_input),
              headers={"Content-type": "application/json"})
response_json = json.loads(response.text)
print('Predicted income: '+response_json[0])

Predicted income: <=50K
